## Step 1
### Produce skims
This step is done before running this notebook. Godspeed, lady.

In [ ]:
cmd = "sh scripts/6b_scripts/submit_all_6b_signal.sh"
cmd = 'parallel -j 4 "python scripts/file_handling/hadd_private.py {}" ::: $(cat sig_files.txt) --eta"'
cmd

'parallel -j 4 "python scripts/file_handling/hadd_private.py {}" ::: $(cat sig_files.txt) --eta"'

In [3]:
import uproot
f = uproot.open("/home/srhelfrich/workarea/CMSSW_12_6_0/src/MultiHiggs/analysis/MultiHAnalysis/output.root")
t = f['sixBtree']
t['bSFshape_central'].array()

<Array [0.562, 0.754, 0.838, ..., 0.796, 0.834, 0.651] type='21 * float32'>

## Step 2
### B tag SFs and corrections

In [ ]:
get_NMSSM(700,400,selection='maxbtag',private=True,suffix='2M'), get_NMSSM()

('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_700_MY_400_2M/ntuple.root',
 '/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/Official_NMSSM/NMSSM_XToYHTo6B_MX-1400_MY-700_TuneCP5_13TeV-madgraph-pythia8/ntuple.root')

In [4]:
# maxbtag = SixB(get_NMSSM(700,400,selection='maxbtag',private=True,suffix='2M'))
selection = SixB(get_NMSSM(1000,400))
selection.bSFshape_central

ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/Official_NMSSM/NMSSM_XToYHTo6B_MX-1000_MY-400_TuneCP5_13TeV-madgraph-pythia8/ntuple.root
model: /eos/uscms/store/user/srosenzw/weaver/models/exp_sixb_official/feynnet_ranker_6b/20230731_7d266883bbfb88fe4e226783a7d1c9db_ranger_lr0.0047_batch2000_withbkg/predict_output/2018/NMSSM_XToYHTo6B_MX-1000_MY-400_TuneCP5_13TeV-madgraph-pythia8.root


<Array [1, 1, 1, 1, 1, 1, 1, ..., 1, 1, 1, 1, 1, 1, 1] type='74663 * float64'>

In [11]:
assert not ak.all(maxbtag.tree['bSFshape_central'].array() == 1)

AssertionError: 

In [ ]:
%run scripts/calculate_2d_sf_corrs.py

## Step 3
### FeynNet Predictions & Efficiency Evaluation
FeynNet predictions are done before running this notebook. Efficiencies are evaluated here.